In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import requests
import re

In [24]:
driver_path = "./chromedriver"
url = "https://kokkai.sugawarataku.net/giin/rl049.html"

In [27]:
def zimin_clawling(driver_path, url):
    """
    driver_path : google driverが置かれているpath
    url : 取得したいサイトのurl
    """
    # ヘッドレスモードでブラウザを起動
    options = Options()
    options.add_argument("--headless")
    
    # driverの設定
    driver = webdriver.Chrome(driver_path, options=options)
    driver.get(url)
    
    # htmlを取得
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 氏名（ひらがな）、氏名、政党、当選回数、地域を取得
    names_hiragana = soup.find_all("span", class_="zt4")
    name_hiragana_list = [name_hiragana.text for name_hiragana in names_hiragana]
    
    names_kanji = soup.find_all("span", class_="zt5")
    name_kanji_list = [name_kanji.text for name_kanji in names_kanji]
    
    parties = soup.find_all("div", class_="cc2")
    party_list = [party.text for party in parties]
    
    times = soup.find_all("div", class_="cc3")
    times_list = [time.text for time in times]
    
    areas = soup.find_all("div", class_="cc4")
    area_list = [area.text for area in areas]
    
    # 当選した地域とブロックの情報を分ける
    area_list2 = []
    for area in area_list:
        if "ブロック" in area:
            area = re.findall(r"（.*）", area)
            area_list2.append(area[0][1:-1]) # （）を除く
        else:
            area_list2.append(area)
    
    block_list = []
    for area in area_list:
        block = re.findall(r".*ブロック", area)
        if block:
            block_list.append(block)
        else: block_list.append("")
            
    # DataFrameにまとめる
    zimin_df = pd.DataFrame()
    zimin_df["氏名（ひらがな）"] = name_hiragana_list
    zimin_df["氏名"] = name_kanji_list
    zimin_df["所属"] = party_list
    zimin_df["当選回数"] = times_list
    zimin_df["当選区"] = area_list2
    zimin_df["比例ブロック"] = block_list
    
    # csvとして同じディレクトリに保存
    term = soup.find("h1", id="top").text
    zimin_df.to_csv(f"./{term}.csv", index=False, encoding="shift jis")
    return 